# Fill the shared array with data

In [11]:
import numpy as np
import skmap_bindings
import time

In [12]:
n_feat = 6
n_threads = 48
x_off = 2
y_off = 2
x_size = 4000
y_size = 4000

bands_list = [1,] # The first band is indexed as 1

conf_GDAL = {
             "CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_MULTIRANGE": "SINGLE_GET",
             "GDAL_HTTP_CONNECTTIMEOUT": "320",
             "GDAL_HTTP_TIMEOUT": "320",
             "CPL_VSIL_CURL_USE_HEAD": "NO",
             "GDAL_HTTP_VERSION": "1.0",
             "CPL_CURL_GZIP": "NO"}

out_dates = [
        "20090101_20090228",
        "20090501_20090630",
        "20090701_20090831",
        "20090701_20090831",
        "20091101_20091231"]
file_urls = []
for i in range(len(out_dates)):
    file_urls.append(f"http://192.168.1.30:8333/prod-landsat-ard2/120E_46N/agg/swir2_glad.ard2_m_30m_s_{out_dates[i]}_go_epsg.4326_v20230908.tif")
    
perm_vec =  [0, 2, 4, 5, 3] # row indices of the array where each tiff contenct will go (starting from first element of file_urls)

In [13]:
data = np.empty((n_feat,x_size*y_size), dtype=np.float32, order='C')
print("Starting")
start = time.time()
skmap_bindings.parReadFiles(data, n_threads, file_urls, perm_vec, x_off, y_off, x_size, y_size, bands_list, conf_GDAL)
print(f"C++ part done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Starting
C++ part done in 0.27 s
[[255. 255. 255. ... 255. 255. 255.]
 [  0.   0.   0. ...   0.   0.   0.]
 [ 35.  34.  31. ...  42.  41.  41.]
 [255. 255. 255. ... 255. 255. 255.]
 [ 27.  27.  26. ...  44.  43.  43.]
 [ 27.  27.  26. ...  44.  43.  43.]]
(6, 16000000)
